## Method
- Make prediction for localidad level (Teusaquillo) using experimental design 2, Naive Counting and SEPPexp based on week-days model.
- Compare model performance using hit rate traditional implementation, true positives proportion and PAI.

## Hypothesis:
SEPP exp prediction based on week-days performance is better than the obtained with counting model.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
from services.prediction_experiment import PredictionExperiment

Failed to import `rtree`.
Failed to import `rtree`.


In [2]:
from services import prediction_metrics

In [3]:
def get_hit_rate_from_dict(row,column,coverage):
    return row[column][coverage]

## SIEDCO

In [4]:
csv_path = '/Users/anamaria/Desktop/dev/security_project/datasets/deduplicate_siedco_09062020.csv'
siedco_info = {'name':'SIEDCO','path':csv_path}

## Counting

In [5]:
train_dates = {'initial':'2018-03-01','final':'2018-09-30'}
validation_dates = {'initial':'2018-10-01','final':'2018-10-07'}
metrics = ''
aggregation = ''

In [6]:
filter_localidad = {'field':'LOCALIDAD','value':'TEUSAQUILLO'}
model = "NaiveCounting"
localidad_experiment = PredictionExperiment(dataset_info=siedco_info, custom_filter=filter_localidad,train_dates=train_dates, validation_dates=validation_dates, model=model,metrics='',aggregation_data='')
prediction_array = localidad_experiment.run_ncv_experiment(time_unit='',grid_size=150, outer_iterations='')

/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (32,44) have mixed types. Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 9, 30, 0, 0)}
current_validation_date 2018-10-01 00:00:00
381
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 1, 0, 0)}
current_validation_date 2018-10-02 00:00:00
381
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 2, 0, 0)}
current_validation_date 2018-10-03 00:00:00
384


../services/process_data.py:77: UserWarning: Empty filter result, check dates. Initial date: 2018-10-01, final date: 2018-10-02
  warnings.warn('Empty filter result, check dates. Initial date: '+initial_date.strftime('%Y-%m-%d')+', final date: '+real_final_date.strftime('%Y-%m-%d'))


train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 3, 0, 0)}
current_validation_date 2018-10-04 00:00:00
385
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 4, 0, 0)}
current_validation_date 2018-10-05 00:00:00
385
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 5, 0, 0)}
current_validation_date 2018-10-06 00:00:00
387
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 6, 0, 0)}
current_validation_date 2018-10-07 00:00:00


../services/process_data.py:77: UserWarning: Empty filter result, check dates. Initial date: 2018-10-04, final date: 2018-10-05
  warnings.warn('Empty filter result, check dates. Initial date: '+initial_date.strftime('%Y-%m-%d')+', final date: '+real_final_date.strftime('%Y-%m-%d'))
../services/process_data.py:77: UserWarning: Empty filter result, check dates. Initial date: 2018-10-06, final date: 2018-10-07
  warnings.warn('Empty filter result, check dates. Initial date: '+initial_date.strftime('%Y-%m-%d')+', final date: '+real_final_date.strftime('%Y-%m-%d'))


387


In [7]:
df_counting = pd.DataFrame(prediction_array, columns =['initial-date','final-date','prediction','eval_pts'])
df_result = df_counting.copy()

In [8]:
coverages = [2,4,6,8,10,12,14,16,18,20]
df_result['hitrate_default'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'default'), axis=1)
df_result['hitrate_TP'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'ground_truth_coverage'), axis=1)


In [9]:
##For hitrate_TP "true positives"
df_result['coverage_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].keys())[0], axis=1)
df_result['hit_rate_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].values())[0], axis=1)
df_result['PAI_TP'] = df_result['hit_rate_TP'] / (df_result['coverage_TP']/100)

In [10]:
##For hitrate_default
coverages = [2.0,4.0,6.0,8.0,10.0,12.0,14.0,16.0,18.0,20.0]
column_dict = 'hitrate_default'
for c in coverages:
    new_hit_rate_column = 'hit_rate_default_coverage_'+str(c)
    df_result[new_hit_rate_column] = df_result.apply(lambda row: get_hit_rate_from_dict(row,column_dict,c), axis=1)

    ##PAI
    new_column = 'PAI_default_coverage_'+str(c)
    df_result[new_column] = df_result[new_hit_rate_column]/(c/100)


In [11]:
#delete datetimes where no crimes were reported (0 crimes on ground truth -> hit-rate = -1)
df_result = df_result[df_result['hit_rate_default_coverage_2.0']!= -1]
print(df_result.mean())

coverage_TP                        0.126126
hit_rate_TP                        0.000000
PAI_TP                             0.000000
hit_rate_default_coverage_2.0      0.200000
PAI_default_coverage_2.0          10.000000
hit_rate_default_coverage_4.0      0.200000
PAI_default_coverage_4.0           5.000000
hit_rate_default_coverage_6.0      0.200000
PAI_default_coverage_6.0           3.333333
hit_rate_default_coverage_8.0      0.200000
PAI_default_coverage_8.0           2.500000
hit_rate_default_coverage_10.0     0.200000
PAI_default_coverage_10.0          2.000000
hit_rate_default_coverage_12.0     0.200000
PAI_default_coverage_12.0          1.666667
hit_rate_default_coverage_14.0     0.200000
PAI_default_coverage_14.0          1.428571
hit_rate_default_coverage_16.0     0.266667
PAI_default_coverage_16.0          1.666667
hit_rate_default_coverage_18.0     0.266667
PAI_default_coverage_18.0          1.481481
hit_rate_default_coverage_20.0     0.266667
PAI_default_coverage_20.0       

## SEPPexp week-day

In [16]:
train_dates = {'initial':'2014-03-01','final':'2018-09-30'}
validation_dates = {'initial':'2018-10-01','final':'2018-10-07'}
metrics = ''
aggregation = ''

In [17]:
filter_localidad = {'field':'LOCALIDAD','value':'TEUSAQUILLO'}
model = "SEPPexpWeekDay"
localidad_experiment = PredictionExperiment(dataset_info=siedco_info, custom_filter=filter_localidad,train_dates=train_dates, validation_dates=validation_dates, model=model,metrics='',aggregation_data='')
prediction_array = localidad_experiment.run_ncv_experiment(time_unit='',grid_size=150, outer_iterations='')

/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (32,44) have mixed types. Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


train_subset_dates {'initial': datetime.datetime(2014, 3, 1, 0, 0), 'final': datetime.datetime(2018, 9, 30, 0, 0)}
current_validation_date 2018-10-01 00:00:00
omega: 0.0045822012274454775, theta: 0.00784789580478372
244
train_subset_dates {'initial': datetime.datetime(2014, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 1, 0, 0)}
current_validation_date 2018-10-02 00:00:00
omega: 0.012158618230911966, theta: 0.007872317244487882
246
train_subset_dates {'initial': datetime.datetime(2014, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 2, 0, 0)}
current_validation_date 2018-10-03 00:00:00
omega: 0.017537528770544375, theta: 0.011210216601137593
259
train_subset_dates {'initial': datetime.datetime(2014, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 3, 0, 0)}
current_validation_date 2018-10-04 00:00:00
omega: 0.047654581722577305, theta: 0.011914034597373611
252
train_subset_dates {'initial': datetime.datetime(2014, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 4, 0, 0)}
curre

In [18]:
df_seppexp = pd.DataFrame(prediction_array, columns =['initial-date','final-date','prediction','eval_pts'])
df_result = df_seppexp.copy()

In [19]:
coverages = [2,4,6,8,10,12,14,16,18,20]
df_result['hitrate_default'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'default'), axis=1)
df_result['hitrate_TP'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'ground_truth_coverage'), axis=1)


In [20]:
##For hitrate_TP "true positives"
df_result['coverage_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].keys())[0], axis=1)
df_result['hit_rate_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].values())[0], axis=1)
df_result['PAI_TP'] = df_result['hit_rate_TP'] / (df_result['coverage_TP']/100)

In [21]:
##For hitrate_default
coverages = [2.0,4.0,6.0,8.0,10.0,12.0,14.0,16.0,18.0,20.0]
column_dict = 'hitrate_default'
for c in coverages:
    new_hit_rate_column = 'hit_rate_default_coverage_'+str(c)
    df_result[new_hit_rate_column] = df_result.apply(lambda row: get_hit_rate_from_dict(row,column_dict,c), axis=1)

    ##PAI
    new_column = 'PAI_default_coverage_'+str(c)
    df_result[new_column] = df_result[new_hit_rate_column]/(c/100)


In [22]:
#delete datetimes where no crimes were reported (0 crimes on ground truth -> hit-rate = -1)
df_result = df_result[df_result['hit_rate_default_coverage_2.0']!= -1]
print(df_result.mean())

coverage_TP                       0.128566
hit_rate_TP                       0.000000
PAI_TP                            0.000000
hit_rate_default_coverage_2.0     0.133333
PAI_default_coverage_2.0          6.666667
hit_rate_default_coverage_4.0     0.133333
PAI_default_coverage_4.0          3.333333
hit_rate_default_coverage_6.0     0.133333
PAI_default_coverage_6.0          2.222222
hit_rate_default_coverage_8.0     0.133333
PAI_default_coverage_8.0          1.666667
hit_rate_default_coverage_10.0    0.133333
PAI_default_coverage_10.0         1.333333
hit_rate_default_coverage_12.0    0.400000
PAI_default_coverage_12.0         3.333333
hit_rate_default_coverage_14.0    0.400000
PAI_default_coverage_14.0         2.857143
hit_rate_default_coverage_16.0    0.400000
PAI_default_coverage_16.0         2.500000
hit_rate_default_coverage_18.0    0.400000
PAI_default_coverage_18.0         2.222222
hit_rate_default_coverage_20.0    0.400000
PAI_default_coverage_20.0         2.000000
dtype: floa